In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [ ]:
inicio = time.time()

In [ ]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

In [ ]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [ ]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [ ]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [ ]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [ ]:
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough') #Transformando variáveis categóricas em variáveis binárias
X = onehotencoder.fit_transform(X).toarray()

In [ ]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.Input(shape = (316,)),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 1, activation = 'linear')
    ])
    regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])
    return regressor

In [ ]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [ ]:
resultados = cross_val_score(estimator = regressor, X = X, y = y, cv = 5, scoring = 'neg_mean_absolute_error')

In [ ]:
abs(resultados)

In [ ]:
abs(resultados.mean())

In [ ]:
resultados.std()